In [13]:
def print_embeddings_cost(texts):
  import tiktoken
  enc = tiktoken.encoding_for_model('text-embedding-ada-002')
  total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
  print(f'Total Tokens: {total_tokens}')
  print(f'Embeddings consto en dolar:  {total_tokens / 1000 * 0.0001:.5f}')

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key=os.getenv('OPENAI_API_KEY', 'YourAPIKey')

# Paquetes necesarios

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# Carga de documentos

In [3]:
loader = PyPDFLoader(".\Doc\mtg.pdf")
pages = loader.load_and_split()

## Slipt y chucks del documento

In [29]:
# Paquetes necesarios
from langchain.text_splitter import RecursiveCharacterTextSplitter

def create_chunks(doc_to_chunk):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=100,
        length_function=len
        )
    return text_splitter.split_documents(doc_to_chunk)

chunks = create_chunks(pages)

In [30]:
chunks

[Document(page_content='Guía de inicio \nrápidoEdad: 13 o +', metadata={'source': '.\\Doc\\mtg.pdf', 'page': 0}),
 Document(page_content='2Bienvenido a Magic: The Gathering, el mejor juego \nde cartas intercambiables del mundo. Estás a punto \nde descubrir por qué Magic tiene millones de jugadores \nen todo el mundo: la profundidad de su estrategia, la \nbelleza de sus mundos, la fuerza de sus personajes y su \ncomunidad global.\nSi ya tienes algunas cartas de Magic, es el momento \nde sacarlas. Esas cartas representan las \nherramientas que utilizarás para derrotar a \ntus oponentes. Con ellas invocarás temibles \ncriaturas, blandirás objetos y sortilegios \nmísticos y lanzarás asombrosos hechizos. Las cartas son las piezas básicas del \njuego y puedes mezclarlas en incontables \ncombinaciones.\nHay miles de cartas para elegir, por lo', metadata={'source': '.\\Doc\\mtg.pdf', 'page': 1}),
 Document(page_content='juego y puedes mezclarlas en incontables \ncombinaciones.\nHay miles de ca

In [24]:
print_embeddings_cost(chunks)

Total Tokens: 14567
Embeddings consto en dolar:  0.00146


# Motor de emmbeddings

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(api_key=openai_api_key, model="text-embedding-ada-002")

# Creamos nuestro cliente de chroma

## Guardar en el local

In [32]:
db = Chroma.from_documents(chunks, embeddings,  persist_directory="./datamtgOpenAI")

## Cargar base de datos

In [ ]:
db = Chroma(persist_directory="./datamtgOpenAI", embedding_function=embeddings)

## Operacion

### GET Collection

In [ ]:
db._collection.get()

### Delete

In [31]:
db.delete_collection()

# Realizar consulta

In [33]:

query = "Con cuanta vida empiezan los jugadores?"
docs = db.similarity_search_with_score(query)
docs

[(Document(page_content='Jasper SandnerC 214/ 269 2/1Oo2\n™ & © 2014 Wizards of the CoastSP• M15Campo de \nbatallatú \n16 vidas\nrestantes\nyo\n18 vidas \nrestantesCementerio  Biblioteca  \nBiblioteca  Mano\nManoCementerio  \n3Para comenzar el juego, baraja tu mazo, \ntambién conocido como tu biblioteca. Roba una mano de siete cartas y comprueba cuántas tierras tienes. Puedes mirar la línea de texto que hay bajo la ilustración de cada carta para ver de qué tipo de carta se trata. Para este primer juego, si no tienes al menos dos tierras, baraja de nuevo tu mazo (incluyendo tu mano anterior) y roba una mano nueva.\nCada jugador comienza con 20 vidas, \ny cada uno debe llevar la cuenta de su total de vidas de alguna manera (con un dado, lápiz y papel...). ¡Reduce el total de vidas de tu oponente a 0 y ganarás el juego!Comenzar', metadata={'page': 2, 'source': '.\\Doc\\mtg.pdf'}),
  0.34318149586967045),
 (Document(page_content='Siempre que ganes vidas, puedes \nponer un contador +1/+1 so

# Carga del modelo

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

llm = ChatOpenAI(model_name='gpt-3.5-turbo')
chainQA = load_qa_chain(llm, chain_type="stuff")

# Realizar pregunta al QnA

In [38]:
pregunta = "Mi amigo tom es muy fanatico del magic que podria regalarle?"

In [39]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    docs = db.similarity_search(pregunta)
    response = chainQA.run(input_documents=docs, question=pregunta)
    print(f"Respuesta ChatGPT: {response} \n {cb}")

Respuesta ChatGPT: Podrías considerar regalarle un pack de inicio de Magic: The Gathering. Estos packs vienen con un mazo de 60 cartas listo para jugar, lleno de criaturas, tierras y otros hechizos, además de dos sobres de 15 cartas adicionales para que pueda personalizar su mazo. También podrías considerar regalarle una caja para guardar sus cartas, protectores de cartas o incluso cartas sueltas que le falten en su colección. Si no estás seguro de qué le falta o qué le gustaría, también podrías regalarle una tarjeta de regalo de una tienda de juegos o de Magic para que él pueda elegir lo que más le convenga. 
 Tokens Used: 930
	Prompt Tokens: 771
	Completion Tokens: 159
Successful Requests: 1
Total Cost (USD): $0.0014745000000000001


# QnA Con memoria

In [59]:
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

# Creamos el template del promt
template = """Eres un chatbot manteniendo una conversación con un humano.

Segun el Contexto y el Historial de la conversacion, el humano te hara una pregunta y tu deberas responderla de la mejor manera posible, si no sabes la respuesta, puedes decirle que no sabes.

## Contexto
{context}

## Historial de la conversacion
{chat_history}

## Pregunta del Human y respuesta final
Human: {human_input}
AI:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)

# Creamos la memoria
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

In [60]:
chainQnAMemory = load_qa_chain(
    OpenAI(temperature=0), chain_type="stuff", memory=memory, prompt=prompt
)

In [65]:
with get_openai_callback() as cb:
    pregunta = "cuantos jugadores pueden jugar?"
    docs = db.similarity_search(pregunta, 3)
    response = chainQnAMemory({"input_documents": docs, "human_input": pregunta})
    print(f"Callnack: {cb} \n\n")

response


Callnack: Tokens Used: 1435
	Prompt Tokens: 1375
	Completion Tokens: 60
Successful Requests: 1
Total Cost (USD): $0.0287 




{'input_documents': [Document(page_content='juego y puedes mezclarlas en incontables \ncombinaciones.\nHay miles de cartas para elegir, por lo \nque te preguntarás por dónde comenzar. Cada jugador necesita un mazo de al menos 60 cartas; un pack de inicio es una forma ideal de conseguirlo. Estos packs te ofrecen todo cuanto necesitas para empezar a jugar: un mazo de 60 cartas listo para jugar lleno de criaturas, tierras y otros hechizos. También incluye dos sobres de 15 cartas que podrás utilizar para darle al mazo un toque propio.juega \ncon tu \nPRESA  \nIlust. Tyler Jacobson', metadata={'page': 1, 'source': '.\\Doc\\mtg.pdf'}),
  Document(page_content='Jasper SandnerC 214/ 269 2/1Oo2\n™ & © 2014 Wizards of the CoastSP• M15Campo de \nbatallatú \n16 vidas\nrestantes\nyo\n18 vidas \nrestantesCementerio  Biblioteca  \nBiblioteca  Mano\nManoCementerio  \n3Para comenzar el juego, baraja tu mazo, \ntambién conocido como tu biblioteca. Roba una mano de siete cartas y comprueba cuántas tierra